# Fernando Estrada¶
Problem Set 2: Convolutional Neural Networks CS 4143 - Deep Learning

February 21, 2022

# Libraries and Imports


In [1]:
from keras.models import *
from keras.layers import *
from keras import Input
from keras import Sequential
from tensorflow import keras
from keras import metrics

import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import numpy as np                      

from PIL import Image, ImageOps
from matplotlib import pyplot as plt    
import cv2
import math
import sys

In [2]:
import tensorflow as tf

print("nums of GPus: ", len(tf.config.experimental.list_physical_devices('GPU')))

nums of GPus:  1


# Data and Preprocessing

In [3]:
path = 'data.csv'

In [4]:
data = pd.read_csv(path , nrows= 15000)

In [5]:
data

,Files,Label,Centers
0,images/img0.jpg,rectangle,"(341, 265)"
1,images/img1.jpg,triangle,"(383, 199)"
2,images/img2.jpg,rectangle,"(487, 280)"
3,images/img3.jpg,triangle,"(389, 257)"
4,images/img4.jpg,triangle,"(145, 221)"
...,...,...,...
14995,images/img14995.jpg,rectangle,"(259, 227)"
14996,images/img14996.jpg,triangle,"(356, 268)"
14997,images/img14997.jpg,rectangle,"(341, 272)"
14998,images/img14998.jpg,triangle,"(355, 107)"


In [6]:
images = [ np.asarray(ImageOps.grayscale(Image.open(x)), dtype = 'float32')/255  for x in data['Files'] ]

In [7]:
Y = data.iloc[ : , 1:2 ]

enc = OneHotEncoder()

Y = enc.fit_transform(Y).toarray()

Y = pd.DataFrame(Y)

In [8]:
Y

,0,1,2
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
...,...,...,...
14995,0.0,1.0,0.0
14996,0.0,0.0,1.0
14997,0.0,1.0,0.0
14998,0.0,0.0,1.0


In [9]:
images = np.asarray(images)
images = images.reshape(  len(images), 400, 600, 1 )

In [10]:
x_train , x_test , y_train , y_test = train_test_split(images,Y, test_size=0.2)

# Model

In [11]:
model = Sequential()

model.add( Input(shape=(400,600 ,1), dtype="float32"   )  )
model.add(Conv2D(4, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(4, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(4, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(4, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(4, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(4, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add( Dense(3, activation='sigmoid')  )


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 398, 598, 4)       40        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 199, 299, 4)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 197, 297, 4)       148       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 148, 4)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 146, 4)        148       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 73, 4)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 71, 4)         1

In [115]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy' , 'Recall' , 'Precision'])

# Training

In [116]:
model.fit( x_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

Epoch 1/10
169/169 [==============================] - 18s 90ms/step - loss: 1.0820 - accuracy: 0.3746 - recall: 0.4792 - precision: 0.3762 - val_loss: 0.8907 - val_accuracy: 0.5783 - val_recall: 0.6442 - val_precision: 0.5569
Epoch 2/10
169/169 [==============================] - 14s 80ms/step - loss: 0.8740 - accuracy: 0.5903 - recall: 0.6189 - precision: 0.5748 - val_loss: 0.7128 - val_accuracy: 0.6767 - val_recall: 0.6900 - val_precision: 0.6645
Epoch 3/10
169/169 [==============================] - 14s 81ms/step - loss: 0.6463 - accuracy: 0.7307 - recall: 0.7353 - precision: 0.6759 - val_loss: 0.4017 - val_accuracy: 0.8633 - val_recall: 0.8800 - val_precision: 0.7021
Epoch 4/10
169/169 [==============================] - 14s 81ms/step - loss: 0.3214 - accuracy: 0.8972 - recall: 0.9109 - precision: 0.6975 - val_loss: 0.2591 - val_accuracy: 0.9208 - val_recall: 0.9408 - val_precision: 0.7065
Epoch 5/10
169/169 [==============================] - 14s 81ms/step - loss: 0.2041 - accuracy: 0

# Evaluation

In [14]:
model = keras.models.load_model('ShapeDetectMoreDataGpuV2')


In [15]:
model.evaluate(x_test,y_test)

94/94 [==============================] - 8s 18ms/step - loss: 0.0670 - accuracy: 0.9757 - recall: 0.9067 - precision: 0.8326


[0.06701995432376862,
 0.9756666421890259,
 0.9066666960716248,
 0.8325681090354919]

In [118]:
model.save('ShapeDetectMoreDataGpuV2')

INFO:tensorflow:Assets written to: ShapeDetectMoreDataGpuFilter\assets


In [11]:
modelCheck = keras.models.load_model('ShapeDetectMoreDataGpuV2')


In [18]:
modelCheck.evaluate(x_test,y_test)

94/94 [==============================] - 2s 21ms/step - loss: 0.0573 - accuracy: 0.9793 - recall: 0.9073 - precision: 0.8422


[0.05726794898509979,
 0.9793333411216736,
 0.9073333144187927,
 0.8422029614448547]

In [12]:
modelCheck.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 398, 598, 4)       40        
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 199, 299, 4)       0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 197, 297, 4)       148       
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 98, 148, 4)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 96, 146, 4)        148       
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 48, 73, 4)         0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 46, 71, 4)       

# Prediction

In [16]:
def get_pred(arg1):
    modelTest = keras.models.load_model('ShapeDetectMoreDataGpuV1')

    image1 = np.asarray(ImageOps.grayscale(Image.open(arg1)), dtype = 'float32')/255
    
    image1 = np.asarray(image1)
    image1 = image1.reshape(  1, 400, 600, 1 )        
    
    
    result = modelTest.predict(image1)
        
    tClass = result.max()
    
    if(tClass == result[0][0]):
        
        print('Circle')

    if(tClass == result[0][1]):
        
        print('Rectangle')
        
    if(tClass == result[0][2]):
        
        print('Triangle')    

        
    print(result)

In [17]:
get_pred("images/img49999.jpg")

Circle
[[9.9985993e-01 9.4094422e-24 9.8826563e-01]]
